<a href="https://colab.research.google.com/github/danielopezr/COMPETENCIA_AI4ENG_UDEA_2025-2/blob/main/03%20-%20procesado%20completo%20y%20Random%20Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importaciones

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve


import math
import unicodedata

# Cargando los archivos

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 2.85GB/s]


In [ ]:
!unzip udea*.zip > /dev/null

# Procesando el Train

## Funciones para procesar datos

In [ ]:
def deleteColumns(df, columns):
    """
    Elimina las columnas especificadas de un DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columns (list): Lista de nombres de columnas a eliminar.

    Returns:
        None: elimina las co
    """

    df.drop(columns, axis=1, inplace=True)

In [ ]:
def fillDefault(df, column, value):
    """
    Rellena los valores nulos de una columna en un DataFrame con un valor por defecto.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.
        value (any): Valor con el que se reemplazarán los nulos.

    Returns:
        None: Modifica el DataFrame directamente.
    """
    df[column] = df[column].fillna(value)

In [ ]:
def fillColumnsDefaults(df, columnsDefaults):
  """
    Rellena los valores nulos de multiples columnas en un DataFrame con valores valor
    por defecto de cada columna.

     Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columnsDefaults (str, str): Diccionario que contiene como clave las columnas
        a modificar y como valor el valor con el que se reemplazarán los nulos.

    Returns:
        None: Modifica el DataFrame directamente.

  """
  for column in columnsDefaults:
    value = columnsDefaults[column]
    fillDefault(df, column, value)

In [ ]:
def replaceNullByList(df, column, values):
    """
    Reemplaza los valores nulos de una columna en un DataFrame con valores aleatorios de una lista dada.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.
        values (list): Lista de valores posibles para reemplazar los nulos.

    Returns:
        None: Modifica el DataFrame directamente.
    """
    numNulls = df[column].isna().sum()
    randomReplaces = np.random.choice(values, size=numNulls, replace=True)
    df.loc[df[column].isna(), column] = randomReplaces

In [ ]:
def replacesColumnsNullsByLists(df, columnsLists):
    """
    Reemplaza los valores nulos de varias columna especificadas de un DataFrame con valores aleatorios
    de listas dadas para cada columna.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columnsLists (str, list): Dicionario con clave las columnas a modificar y como valor
        una lista de valores posibles para reemplazar los nulos.

    Returns:
        None: Modifica el DataFrame directamente.
    """

    for column in columnsLists:
        values = columnsLists[column]
        replaceNullByList(df, column, values)

In [ ]:
def getDistincValues(df, column):
    """
    Obtiene una lista de valores únicos no nulos de una columna en un DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a analizar.

    Returns:
        list: Lista de valores distintos sin incluir nulos.
    """
    return df[column].dropna().unique().tolist()

In [ ]:
def replaceNullByColumnsValues(df, column):
    """
    Reemplaza los valores nulos aleatoriamente por valores no nulos de la columna.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.

    Returns:
        None: Modifica el DataFrame directamente.
    """

    values = getDistincValues(df, column)
    numNulls = df[column].isna().sum()
    randomReplaces = np.random.choice(values, size=numNulls, replace=True)
    df.loc[df[column].isna(), column] = randomReplaces

In [ ]:
def replacesNullsColumnsByColumnsValues(df, columns):
    """
    Reemplaza los valores nulos de las columnas aleatoriamente por valores no nulos
    de las columnas ingresadas.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columns (list): lista con los nombres de las columnas a modificar.

    Returns:
        None: Modifica el DataFrame directamente.
    """
    for column in columns:
        replaceNullByColumnsValues(df, column)

In [ ]:
def onehotEncode(df, column):
    """
    Aplica codificación one-hot manual a una columna categórica de un DataFrame.

    Crea nuevas columnas binarias (0 o 1) por cada valor único en la columna original,
    y luego elimina la columna original del DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a codificar.

    Returns:
        pandas.DataFrame: DataFrame con las nuevas columnas codificadas y sin la columna original.
    """
    values = df[column].unique().tolist()

    for value in values:
        df[column + '_' + value] = (df[column] == value).astype(int)

    df.drop(columns=[column], inplace=True)
    return df

In [ ]:
def onehotEncodeColumns(df, columns):
    """
      Aplica codificación one-hot manual a las columnas ingresadas de un DataFrame.

      Crea nuevas columnas binarias (0 o 1) por cada valor único en la columna original,
      y luego elimina la columna original del DataFrame.

      Args:
          df (pandas.DataFrame): DataFrame que contiene los datos.
          columns (list): Nombre de las columnas a codificar.

      Returns:
          pandas.DataFrame: DataFrame con las nuevas columnas codificadas y sin la columna original.
    """
    for column in columns:
        df = onehotEncode(df, column)
    return df

In [ ]:
def formatText(df, column):
    """
    Elimina los acentos de los valores de texto en una columna de un DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a formatear.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    def removeAccents(text):
        if isinstance(text, str):
            return ''.join(
                c for c in unicodedata.normalize('NFKD', text)
                if not unicodedata.combining(c)
            )
        return text

    df[column] = df[column].apply(removeAccents)

In [ ]:
def valuesPercentage(df, column):
    """
    Calcula el porcentaje de frecuencia de cada valor en una columna de un DataFrame.

    Aplica la función `formatText` para eliminar acentos antes del cálculo.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a analizar.

    Returns:
        pandas.Series: Serie con los porcentajes de aparición de cada valor.
    """
    df = df.copy()
    formatText(df, column)
    percentages = (df[column].value_counts(normalize=True) * 100).round(2)
    return percentages

In [ ]:
def valuesFilterSignificance(df, column, min_percentage, printInfo=True):
    """
    Filtra los valores de una columna según un porcentaje mínimo de aparición.

    Usa `valuesPercentage` para calcular la frecuencia relativa de cada valor y
    conserva solo aquellos que superen el umbral especificado.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a analizar.
        min_percentage (float): Porcentaje mínimo requerido para conservar un valor.
        printInfo (bool, opcional): Si es True, muestra información sobre los valores filtrados.
            Por defecto True.

    Returns:
        pandas.Series: Serie con los valores y sus porcentajes que cumplen el criterio.
    """
    percentages = valuesPercentage(df, column)
    filtered = percentages[percentages >= min_percentage]

    if printInfo:
        print(filtered)
        print("Porcentaje acumulado: ", filtered.sum())
        print("Número de valores: ", filtered.shape[0])

    return filtered

In [ ]:
def groupValuesByPercentage(df, column, min_percentage, valueGroup='Otros'):
    """
    Agrupa en una misma categoría los valores de una columna cuyo porcentaje de aparición
    sea menor al umbral especificado.

    Utiliza `valuesFilterSignificance` para identificar los valores significativos
    y reemplaza los menos frecuentes por una etiqueta genérica (por defecto 'Otros').

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a procesar.
        min_percentage (float): Porcentaje mínimo para considerar un valor como significativo.
        valueGroup (str, opcional): Nombre de la categoría donde se agruparán los valores poco frecuentes.
            Por defecto 'Otros'.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    formatText(df, column)
    filtered = valuesFilterSignificance(df, column, min_percentage, printInfo=False)
    values_to_keep = filtered.index.tolist()
    df.loc[~df[column].isin(values_to_keep), column] = valueGroup

In [ ]:
def groupValuesByList(df, column, valuesToGroup, valueGroup='Otros'):
    """
    Agrupa en una misma categoría los valores de una columna que pertenezcan a una lista dada.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.
        valuesToGroup (list): Lista de valores que serán reemplazados por la categoría genérica.
        valueGroup (str, opcional): Nombre de la categoría donde se agruparán los valores especificados.
            Por defecto 'Otros'.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    df.loc[df[column].isin(valuesToGroup), column] = valueGroup

In [ ]:
def groupsValuesColumnsByLists(df, columnsValuesToGroup):
    """
    Agrupa en una misma categoría los valores de las columnas dadas
    que pertenezcan a las respectivas listas dadas.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columnsValuesToGroup (str, tuple): Diccionario que tiene como clave el nombre de la columna a modificar
        y como valor una tupla con una lista con los valores que serán reemplazados por la categoría genérica
        y como segundo elemento de la tupla el nombre del valor en que se agruparan
        Ejemplo:
        {'column_name_1':( ['value1','value2','value3'] , 'Other1' ),
        'column_name_2':( ['value1', 'value2'] , 'Other2' ),
        'column_name_3':( ['value1','value2','value3'] , 'Other3' )}

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    for column in columnsValuesToGroup:
        valuesToGroup = columnsValuesToGroup[column][0]
        groupValue = columnsValuesToGroup[column][1]
        groupValuesByList(df, column, valuesToGroup, groupValue)

In [ ]:
def replaceBinaryValues(oneValue, zeroValue, df, column):
    """
    Reemplaza valores de una columna por equivalentes binarios (1 y 0).

    Args:
        oneValue (any): Valor que se reemplazará por 1.
        zeroValue (any): Valor que se reemplazará por 0.
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    df[column] = df[column].replace({oneValue: 1, zeroValue: 0})
    df[column] = df[column].astype(int)

In [ ]:
def replaceBinaryValuesColumns(df, columnsValues):
    """
    Reemplaza valores de una columna por equivalentes binarios (1 y 0).

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columnsValues (str, tuple): Diccionario que tiene como clave el nombre de la columna a modificar
        y como valor una tupla con los valores que se reemplazarán por 1 y 0 respectivamente.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """

    for column in columnsValues:
        oneValue = columnsValues[column][0]
        zeroValue = columnsValues[column][1]
        replaceBinaryValues(oneValue, zeroValue, df, column)

In [ ]:
def replaceDiscreteValues(replaceDict, df, column):
    """
    Reemplaza valores discretos en una columna según un diccionario de mapeo.

    Args:
        replaceDict (dict): Diccionario que define los reemplazos.
            Ejemplo: {'A': 'Alto', 'B': 'Bajo'}.
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    df[column] = df[column].replace(replaceDict)

In [ ]:
def replaceDiscreteValuesColumns(df, columnsValues):
    """
    Reemplaza valores discretos en una columna según un diccionario de mapeo.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columnsValues (str, dict): Diccionario que tiene como clave el nombre de la columna a modificar
        y como valor un diccionario con los valores que se reemplazarán por sus equivalentes.

        Ejemplo del diccionario de valores: {'A': 'Alto', 'B': 'Bajo'}.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """

    for column in columnsValues:
        replaceDict = columnsValues[column]
        replaceDiscreteValues(replaceDict, df, column)

In [ ]:
traindf = pd.read_csv("train.csv")
traindf.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
traindf.columns

Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4'],
      dtype='object')

## Procesando los datos

In [ ]:
def procesDataFrame(df, columnsToDelete, columnsDefaults, columnsNullsList, columnsNullsValues,
                    relativeFrecuencytoGrouping, columnsListsReplacesValues, onehotColumns,
                    columnsBinaryValues, discreteColumnsValues):
  ###################### Eliminando columnas ######################

  #Eliminando columna duplicada
  deleteColumns(df, ['F_TIENEINTERNET.1'])

  #Eliminando columnas irrelevantes
  deleteColumns(df, columnsToDelete)

  ###################### Eliminando nulos ######################

  #Cambiando nulos a valores por defecto
  fillColumnsDefaults(df, columnsDefaults)

  #cambiando nulos por valores de una lista
  replacesColumnsNullsByLists(df, columnsNullsList)

  #cambiando nulos por valores de las columnas
  replacesNullsColumnsByColumnsValues(df, columnsNullsValues)

  ###################### Conversión a valores numericos ######################

  #Arreglando la columna de programas academicos
  groupValuesByPercentage(df, 'E_PRGM_ACADEMICO', relativeFrecuencytoGrouping, 'OTROS')
  df = onehotEncode(df, 'E_PRGM_ACADEMICO')

  #Agrupando valores similares de columnas
  groupsValuesColumnsByLists(df, columnsListsReplacesValues)

  #One hot encoding
  df = onehotEncodeColumns(df, onehotColumns)

  #Convirtiendo valores binarios
  replaceBinaryValuesColumns(df, columnsBinaryValues)

  #Reemplazar valores discretos con orden
  replaceDiscreteValuesColumns(df, discreteColumnsValues)

  return df

In [ ]:
procesDataFrame(
    df = traindf,
    columnsToDelete = ['E_PRIVADO_LIBERTAD', 'F_TIENELAVADORA', 'ID'],
    columnsDefaults = {
      'E_VALORMATRICULAUNIVERSIDAD': "No pagó matrícula",
      'E_HORASSEMANATRABAJA': '0',
      'F_TIENEINTERNET': 'No',
      'E_PAGOMATRICULAPROPIO': "No",
      'F_TIENECOMPUTADOR': "No"
    },
    columnsNullsList = {},
    columnsNullsValues = ['F_ESTRATOVIVIENDA', 'F_EDUCACIONPADRE', 'F_TIENEAUTOMOVIL', 'F_EDUCACIONMADRE'],
    relativeFrecuencytoGrouping = 0.2,
    columnsListsReplacesValues = {
        'F_EDUCACIONPADRE':(['Ninguno', 'No sabe','No Aplica'], 'Ninguno'),
        'F_EDUCACIONMADRE':(['Ninguno', 'No sabe','No Aplica'], 'Ninguno')
    },
    onehotColumns = ['E_PRGM_DEPARTAMENTO', 'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE'],
    columnsBinaryValues = {
        'F_TIENEINTERNET': ('Si', 'No'),
        'F_TIENEAUTOMOVIL': ("Si", "No"),
        'E_PAGOMATRICULAPROPIO': ("Si", "No"),
        'F_TIENECOMPUTADOR' : ("Si", "No")
    },
    discreteColumnsValues = {
        'E_VALORMATRICULAUNIVERSIDAD':{
                'No pagó matrícula': 0,
                'Menos de 500 mil': 1,
                'Entre 500 mil y menos de 1 millón': 2,
                'Entre 1 millón y menos de 2.5 millones': 3,
                'Entre 2.5 millones y menos de 4 millones': 4,
                'Entre 4 millones y menos de 5.5 millones': 5,
                'Entre 5.5 millones y menos de 7 millones': 6,
                'Más de 7 millones': 7
        },
        'E_HORASSEMANATRABAJA': {
                '0':0,
                'Menos de 10 horas':1,
                'Entre 11 y 20 horas':2,
                'Entre 21 y 30 horas':3,
                'Más de 30 horas':4
        },
        'F_ESTRATOVIVIENDA':{
                'Sin Estrato': 0,
                'Estrato 1': 1,
                'Estrato 2': 2,
                'Estrato 3': 3,
                'Estrato 4': 4,
                'Estrato 5': 5,
                'Estrato 6': 6
        },
        'PERIODO_ACADEMICO':{
                20183: 0,
                20184: 1,
                20194: 2,
                20195: 3,
                20196: 4,
                20202: 5,
                20203: 6,
                20212: 7,
                20213: 8
        },
        'RENDIMIENTO_GLOBAL':{
                'bajo': 1,
                'medio-bajo': 2,
                'medio-alto': 3,
                'alto': 4
        }
    }
)


/tmp/ipython-input-2701605062.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column + '_' + value] = (df[column] == value).astype(int)
/tmp/ipython-input-2701605062.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column + '_' + value] = (df[column] == value).astype(int)
/tmp/ipython-input-2701605062.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,RENDIMIENTO_GLOBAL,INDICADOR_1,...,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Técnica o tecnológica incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Educación profesional incompleta,F_EDUCACIONMADRE_Educación profesional completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Ninguno
0,7,6,1,3,1,1,0,1,3,0.322,...,1,0,0,0,0,0,0,0,0,0
1,7,4,0,3,0,0,0,1,1,0.311,...,0,1,0,0,0,0,0,0,0,0
2,6,4,4,3,1,0,0,0,1,0.297,...,0,0,1,0,0,0,0,0,0,0
3,3,5,0,4,1,0,0,1,4,0.485,...,0,0,1,0,0,0,0,0,0,0
4,7,4,3,3,1,1,0,1,2,0.316,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,3,2,2,2,1,0,1,1,3,0.237,...,0,0,0,0,0,1,0,0,0,0
692496,7,4,4,3,1,0,0,1,1,0.314,...,0,0,0,0,0,1,0,0,0,0
692497,0,3,1,3,1,0,1,1,2,0.286,...,0,0,0,0,0,1,0,0,0,0
692498,3,4,1,1,0,0,1,1,1,0.132,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
print(traindf)

        PERIODO_ACADEMICO  E_VALORMATRICULAUNIVERSIDAD  E_HORASSEMANATRABAJA  \
0                       7                            6                     1   
1                       7                            4                     0   
2                       6                            4                     4   
3                       3                            5                     0   
4                       7                            4                     3   
...                   ...                          ...                   ...   
692495                  3                            2                     2   
692496                  7                            4                     4   
692497                  0                            3                     1   
692498                  3                            4                     1   
692499                  3                            6                     4   

        F_ESTRATOVIVIENDA  F_TIENEINTER

# Entrenar el modelo

Para esta entrega se utilizo un random forest, encontramos que el score es mayor si se tienen 400 arboles y si cada arbol tiene maximo 30 Ramas, vimos que al aumentar las ramas se genera overfitting

In [ ]:
X = traindf.drop('RENDIMIENTO_GLOBAL', axis=1)
y = traindf['RENDIMIENTO_GLOBAL']

In [ ]:
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=0.3)

In [ ]:
Xtr.shape, Xts.shape, ytr.shape, yts.shape

((484750, 131), (207750, 131), (484750,), (207750,))

In [ ]:
estimator = RandomForestClassifier(n_estimators=500, max_depth=30)

In [ ]:
estimator.fit(Xtr, ytr)

RandomForestClassifier(max_depth=30, n_estimators=400)

In [ ]:
estimator.score(Xts, yts)

0.4139783393501805

# Procesando el Test

In [ ]:
testdf = pd.read_csv("test.csv")
testdf.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


In [ ]:
id_test = testdf['ID']

In [ ]:
procesDataFrame(
    df = testdf,
    columnsToDelete = ['E_PRIVADO_LIBERTAD', 'F_TIENELAVADORA', 'ID'],
    columnsDefaults = {
      'E_VALORMATRICULAUNIVERSIDAD': "No pagó matrícula",
      'E_HORASSEMANATRABAJA': '0',
      'F_TIENEINTERNET': 'No',
      'E_PAGOMATRICULAPROPIO': "No",
      'F_TIENECOMPUTADOR': "No"
    },
    columnsNullsList = {},
    columnsNullsValues = ['F_ESTRATOVIVIENDA', 'F_EDUCACIONPADRE', 'F_TIENEAUTOMOVIL', 'F_EDUCACIONMADRE'],
    relativeFrecuencytoGrouping = 0.2,
    columnsListsReplacesValues = {
        'F_EDUCACIONPADRE':(['Ninguno', 'No sabe','No Aplica'], 'Ninguno'),
        'F_EDUCACIONMADRE':(['Ninguno', 'No sabe','No Aplica'], 'Ninguno')
    },
    onehotColumns = ['E_PRGM_DEPARTAMENTO', 'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE'],
    columnsBinaryValues = {
        'F_TIENEINTERNET': ('Si', 'No'),
        'F_TIENEAUTOMOVIL': ("Si", "No"),
        'E_PAGOMATRICULAPROPIO': ("Si", "No"),
        'F_TIENECOMPUTADOR' : ("Si", "No")
    },
    discreteColumnsValues = {
        'E_VALORMATRICULAUNIVERSIDAD':{
                'No pagó matrícula': 0,
                'Menos de 500 mil': 1,
                'Entre 500 mil y menos de 1 millón': 2,
                'Entre 1 millón y menos de 2.5 millones': 3,
                'Entre 2.5 millones y menos de 4 millones': 4,
                'Entre 4 millones y menos de 5.5 millones': 5,
                'Entre 5.5 millones y menos de 7 millones': 6,
                'Más de 7 millones': 7
        },
        'E_HORASSEMANATRABAJA': {
                '0':0,
                'Menos de 10 horas':1,
                'Entre 11 y 20 horas':2,
                'Entre 21 y 30 horas':3,
                'Más de 30 horas':4
        },
        'F_ESTRATOVIVIENDA':{
                'Sin Estrato': 0,
                'Estrato 1': 1,
                'Estrato 2': 2,
                'Estrato 3': 3,
                'Estrato 4': 4,
                'Estrato 5': 5,
                'Estrato 6': 6
        },
        'PERIODO_ACADEMICO':{
                20183: 0,
                20184: 1,
                20194: 2,
                20195: 3,
                20196: 4,
                20202: 5,
                20203: 6,
                20212: 7,
                20213: 8
        }
    }
)


/tmp/ipython-input-2701605062.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column + '_' + value] = (df[column] == value).astype(int)
/tmp/ipython-input-2701605062.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column + '_' + value] = (df[column] == value).astype(int)
/tmp/ipython-input-2701605062.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,INDICADOR_1,INDICADOR_2,...,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Educación profesional incompleta,F_EDUCACIONMADRE_Ninguno,F_EDUCACIONMADRE_Educación profesional completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,0,1,1,3,1,0,1,1,0.328,0.219,...,1,0,0,0,0,0,0,0,0,0
1,6,4,3,2,1,0,0,1,0.227,0.283,...,0,1,0,0,0,0,0,0,0,0
2,7,3,0,3,1,0,0,1,0.285,0.228,...,0,0,1,0,0,0,0,0,0,0
3,3,3,3,1,0,0,0,0,0.160,0.408,...,0,0,0,1,0,0,0,0,0,0
4,7,4,2,2,1,0,0,1,0.209,0.283,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,3,4,4,1,1,1,1,1,0.168,0.410,...,0,0,0,1,0,0,0,0,0,0
296782,0,3,0,4,1,0,0,1,0.471,0.184,...,0,0,0,0,0,0,0,0,1,0
296783,7,4,4,3,1,0,0,1,0.292,0.249,...,0,0,0,0,0,0,0,0,1,0
296784,3,6,2,3,1,0,0,1,0.305,0.219,...,0,0,0,0,0,0,0,0,1,0


## Prediciendo la variable

In [ ]:
Xf = testdf

In [ ]:
print(Xf.columns)

Index(['PERIODO_ACADEMICO', 'E_VALORMATRICULAUNIVERSIDAD',
       'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET',
       'F_TIENEAUTOMOVIL', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR',
       'INDICADOR_1', 'INDICADOR_2',
       ...
       'F_EDUCACIONMADRE_Primaria completa',
       'F_EDUCACIONMADRE_Técnica o tecnológica completa',
       'F_EDUCACIONMADRE_Secundaria (Bachillerato) completa',
       'F_EDUCACIONMADRE_Primaria incompleta', 'F_EDUCACIONMADRE_Postgrado',
       'F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta',
       'F_EDUCACIONMADRE_Educación profesional incompleta',
       'F_EDUCACIONMADRE_Ninguno',
       'F_EDUCACIONMADRE_Educación profesional completa',
       'F_EDUCACIONMADRE_Técnica o tecnológica incompleta'],
      dtype='object', length=131)


In [ ]:
print(Xts.columns)

Index(['PERIODO_ACADEMICO', 'E_VALORMATRICULAUNIVERSIDAD',
       'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET',
       'F_TIENEAUTOMOVIL', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR',
       'INDICADOR_1', 'INDICADOR_2',
       ...
       'F_EDUCACIONMADRE_Postgrado',
       'F_EDUCACIONMADRE_Técnica o tecnológica incompleta',
       'F_EDUCACIONMADRE_Secundaria (Bachillerato) completa',
       'F_EDUCACIONMADRE_Primaria completa',
       'F_EDUCACIONMADRE_Técnica o tecnológica completa',
       'F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta',
       'F_EDUCACIONMADRE_Educación profesional incompleta',
       'F_EDUCACIONMADRE_Educación profesional completa',
       'F_EDUCACIONMADRE_Primaria incompleta', 'F_EDUCACIONMADRE_Ninguno'],
      dtype='object', length=131)


In [ ]:
order_columns = list(Xts.columns)

In [ ]:
Xf[order_columns] # Se cambia el orden de las columnas al mismo del de train para que el modelo las interprete correctamente

,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,INDICADOR_1,INDICADOR_2,...,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Técnica o tecnológica incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Educación profesional incompleta,F_EDUCACIONMADRE_Educación profesional completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Ninguno
0,0,1,1,3,1,0,1,1,0.328,0.219,...,0,0,0,1,0,0,0,0,0,0
1,6,4,3,2,1,0,0,1,0.227,0.283,...,0,0,0,0,1,0,0,0,0,0
2,7,3,0,3,1,0,0,1,0.285,0.228,...,0,0,1,0,0,0,0,0,0,0
3,3,3,3,1,0,0,0,0,0.160,0.408,...,0,0,0,0,0,0,0,0,1,0
4,7,4,2,2,1,0,0,1,0.209,0.283,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,3,4,4,1,1,1,1,1,0.168,0.410,...,0,0,0,0,0,0,0,0,1,0
296782,0,3,0,4,1,0,0,1,0.471,0.184,...,0,0,0,0,0,0,0,1,0,0
296783,7,4,4,3,1,0,0,1,0.292,0.249,...,0,0,0,0,0,0,0,1,0,0
296784,3,6,2,3,1,0,0,1,0.305,0.219,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
yf = estimator.predict(testdf[order_columns])

In [ ]:
yf

array([4, 3, 4, ..., 3, 4, 4])

## Creando la submision

In [ ]:
submission = pd.DataFrame({
    "ID": id_test,
    "RENDIMIENTO_GLOBAL": yf
})


In [ ]:
submission

,ID,RENDIMIENTO_GLOBAL
0,550236,4
1,98545,3
2,499179,4
3,782980,1
4,785185,1
...,...,...
296781,496981,1
296782,209415,4
296783,239074,3
296784,963852,4


In [ ]:
format_dict = {
  1 :'bajo',
  2 :'medio-bajo',
  3 :'medio-alto',
  4 :'alto'
}

replaceDiscreteValues(format_dict, submission, 'RENDIMIENTO_GLOBAL')

In [ ]:
submission

,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo
...,...,...
296781,496981,bajo
296782,209415,alto
296783,239074,medio-alto
296784,963852,alto


In [ ]:
submissionb_example = pd.read_csv('submission_example.csv')

In [ ]:
submissionb_example

,ID,RENDIMIENTO_GLOBAL
0,550236,medio-bajo
1,98545,medio-bajo
2,499179,medio-alto
3,782980,alto
4,785185,medio-bajo
...,...,...
296781,496981,alto
296782,209415,medio-alto
296783,239074,medio-alto
296784,963852,medio-bajo


In [ ]:
submission.to_csv('mysubmission.csv', index=False)